<a href="https://colab.research.google.com/github/yt-oh96/Lab_tutorial/blob/master/Task1_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
ori_image = cv2.imread('image1.tif',1)

plt.subplot(121)
plt.imshow(ori_image, cmap='gray')
plt.title('ori_image')

plt.show()


In [0]:
#3
M = np.float32([[1,0,28],[0,1,60]])
translate_image = cv2.warpAffine(ori_image, M, ori_image.shape[:2])

plt.subplot(121)
plt.imshow(ori_image, cmap='gray')
plt.title('ori_image' )

plt.subplot(122)
plt.imshow(translate_image, cmap='gray')
plt.title('dis_image')

plt.show()

In [0]:
def SSD(ori, dis):
  err = np.sum((ori - dis) ** 2)
  err /= (ori.shape[0] * ori.shape[1])
 
  return err

print(SSD(ori_image, dis_image))

#for i in range()


In [0]:
def optim(ori, dis, mode='', distortion=''):
  x,y = ori.shape[:2]
  
  error = []
  if distortion == 'translation':
    i=0
    for dx in range(-x,x):
      i+=1
      error_y=[]
      j=0
      for dy in range(-y,y):
        
        j+=1
        M = np.float32([[1,0,dx],[0,1,dy]])
        dst = cv2.warpAffine(ori, M, (x,y))

        if mode == 'ssd':
          err = SSD(dst, dis)
        elif mode == 'mi':
          err = -MI(dst,dis)
        
        error_y.append(err)
        
        print(str(i)+','+str(j)+'/512'+', error : ',err)
      error.append(error_y)

  elif distortion == 'rotation':
    w,h = ori.shape[:2]
    for i in range(360):
      
      rotation_mat = cv2.getRotationMatrix2D((w/2,h/2),i,1)
      rotation_image = cv2.warpAffine(ori, rotation_mat, (w,h))
       
      if mode == 'ssd':
        err = SSD(rotation_image, dis)
      elif mode == 'mi':
        err = -MI(rotation_image,dis)

      print(str(i)+','+'/360'+', error : ',err)
      error.append(err)  
  return error


In [0]:
final = optim(translate_image, ori_image, mode='ssd', distortion='translation')

In [0]:
Z = final
Z_min = np.min(Z, axis=0)
Z_min_idx = np.argmin(Z_min)

Z_min_y = np.min(Z, axis=1)
Z_min_idx_y = np.argmin(Z_min_y)

Z_min_idx_y -= 256
Z_min_idx -= 256

print('min_idx = ',Z_min_idx_y,',',Z_min_idx)

from mpl_toolkits.mplot3d import Axes3D
x = np.linspace(-256,255,512)
y = np.linspace(-256,255,512)
Z_np = np.array(Z)
xx, yy = np.meshgrid(x,y)

plt.figure(figsize=(5,5))
ax = plt.subplot(1,1,1,projection='3d')
ax.plot_surface(xx,yy,Z_np)
#ax.view_init(0,0)
plt.show()

In [0]:
M = np.float32([[1,0,-28],[0,1,-60]])
dst_image = cv2.warpAffine(translate_image, M, translate_image.shape[:2])

plt.subplot(121)
plt.imshow(ori_image, cmap='gray')
plt.title('ori_image')

plt.subplot(122)
plt.imshow(dst_image, cmap='gray')
plt.title('dis_image')

plt.show()

In [0]:
fig, axes = plt.subplots(1,2)
axes[0].hist(ori_image.ravel(), bins=30)
axes[1].hist(translate_image.ravel(), bins=30)

In [0]:
def entropy(image):
  image = np.asarray(image)

  p_image = image / image.sum()

  p_image = p_image[np.nonzero(p_image)]
  return -np.sum(p_image*np.log2(p_image))

def MI(image_A, image_B):
  image_A = np.asarray(image_A.ravel())
  image_B = np.asarray(image_B.ravel())
  print(image_A.shape)

  histogram_A = np.histogram(image_A, 30)[0]
  histogram_B = np.histogram(image_B, 30)[0]
  histogram_AB = np.histogram2d(image_A, image_B, 30)[0]

  H_a = entropy(image_A)
  H_b = entropy(image_B)
  H_ab = entropy(histogram_AB)
  #H_ab = entropy(np.concatenate([image_A, image_B]))
  print(H_a, ' ',H_b,' ', H_ab)
  MI_ab = H_a + H_b - H_ab

  return MI_ab

mi = MI(ori_image, dis_image)
print(mi)

In [0]:
final_mi = optim(translate_image, ori_image, mode='mi', distortion='translation')

In [0]:
import pandas as pd

pd_final = pd.DataFrame(final_mi)
Z_mi = pd_final.fillna(-15)
Z_mi = np.asarray(Z_mi)

Z_mi_min = np.min(Z_mi, axis=0)
Z_mi_min_idx = np.argmin(Z_mi_min)

Z_mi_min_y = np.min(Z_mi, axis=1)
Z_mi_min_idx_y = np.argmin(Z_mi_min_y)

Z_mi_min_idx_y -= 256
Z_mi_min_idx -= 256

print('min_idx = ',Z_mi_min_idx_y,',',Z_mi_min_idx)

In [0]:
from mpl_toolkits.mplot3d import Axes3D
x = np.linspace(-256,255,512)
y = np.linspace(-256,255,512)
Z_np = np.array(Z_mi)
xx, yy = np.meshgrid(x,y)

plt.figure(figsize=(5,5))
ax = plt.subplot(1,1,1,projection='3d')
ax.plot_surface(xx,yy,Z_np)
ax.view_init(30,30)
plt.show()

In [0]:
hist_2d, x_edges, y_edges = np.histogram2d(ori_image.ravel(), translate_image.ravel(), bins=50)
plt.imshow(hist_2d.T, origin='lower')

In [0]:
M = np.float32([[1,0,-28],[0,1,-60]])
dst_image = cv2.warpAffine(translate_image, M, translate_image.shape[:2])

hist_2d, x_edges, y_edges = np.histogram2d(dst_image.ravel(), ori_image.ravel(), bins=50)
plt.imshow(hist_2d.T, origin='lower')

In [0]:
plt.subplot(121)
plt.imshow(ori_image, cmap='gray')
plt.title('ori_image' )

plt.subplot(122)
plt.imshow(dst_image, cmap='gray')
plt.title('dst_image')

plt.show()

In [0]:
###3_2
w,h = ori_image.shape[:2]
rotation_mat = cv2.getRotationMatrix2D((w/2,h/2),30,1)
rotation_image = cv2.warpAffine(ori_image, rotation_mat, ori_image.shape[:2])

plt.subplot(121)
plt.imshow(ori_image, cmap='gray')
plt.title('ori_image')

plt.subplot(122)
plt.imshow(rotation_image, cmap='gray')
plt.title('rotation_image')

plt.show()

In [0]:
final = optim( rotation_image,ori_image, mode='ssd', distortion='rotation')

In [0]:
Z = final
Z_min = np.argmin(Z, axis=0)

print('min_rotation = ',Z_min)

from mpl_toolkits.mplot3d import Axes3D
x = np.linspace(0,360,360)
Z_np = np.array(Z)

plt.figure(figsize=(5,5))
plt.plot(x,Z_np)
plt.show()

In [0]:
rotation_mat = cv2.getRotationMatrix2D((w/2,h/2),330,1)
dst_image = cv2.warpAffine(rotation_image, rotation_mat, (w,h))

plt.subplot(131)
plt.imshow(ori_image, cmap='gray')
plt.title('ori_image')


plt.subplot(132)
plt.imshow(rotation_image, cmap='gray')
plt.title('rotation_image')


plt.subplot(133)
plt.imshow(dst_image, cmap='gray')
plt.title('dst_image')

plt.show()

In [0]:
fig, axes = plt.subplots(1,2)
axes[0].hist(ori_image.ravel(), bins=30)
axes[1].hist(rotation_image.ravel(), bins=30)

In [0]:
final_mi = optim(rotation_image, ori_image, mode='mi', distortion='rotation')

In [0]:
import pandas as pd

pd_final = pd.DataFrame(final_mi)
Z_mi = pd_final.fillna(15)
Z_mi = np.asarray(Z_mi)

#Z_mi_min = np.min(Z_mi, axis=0)
Z_mi_min_idx = np.argmin(Z_mi)


print('min_idx = ',Z_mi_min_idx)

In [0]:
Z = final_mi
Z_min = np.argmin(Z, axis=0)

print('min_rotation = ',Z_min)

x = np.linspace(0,360,360)
Z_np = np.array(Z)

plt.figure(figsize=(5,5))
plt.plot(x,Z_np)
plt.show()

In [0]:
hist_2d, x_edges, y_edges = np.histogram2d(ori_image.ravel(), rotation_image.ravel(), bins=50)
plt.imshow(hist_2d.T, origin='lower')

In [0]:
rotation_mat = cv2.getRotationMatrix2D((w/2,h/2),330,1)
dst_image = cv2.warpAffine(rotation_image, rotation_mat, (w,h))

hist_2d, x_edges, y_edges = np.histogram2d(dst_image.ravel(), ori_image.ravel(), bins=50)
plt.imshow(hist_2d.T, origin='lower')

In [0]:
plt.subplot(131)
plt.imshow(ori_image, cmap='gray')
plt.title('ori_image')


plt.subplot(132)
plt.imshow(rotation_image, cmap='gray')
plt.title('rotation_image')


plt.subplot(133)
plt.imshow(dst_image, cmap='gray')
plt.title('dst_image')

plt.show()